**Copyright**

Jelen forráskód a Budapesti Műszaki és Gazdaságtudományi Egyetemen tartott
"Deep Learning a gyakorlatban Python és LUA alapon" tantárgy segédanyagaként készült.

A tantárgy honlapja: http://smartlab.tmit.bme.hu/oktatas-deep-learning
Deep Learning kutatás: http://smartlab.tmit.bme.hu/deep-learning

A forráskódot GPLv3 licensz védi. Újrafelhasználás esetén lehetőség szerint kérjük
az alábbi szerzőt értesíteni.

2021 (c) Csapó Tamás Gábor (csapot kukac tmit pont bme pont hu),
Gyires-Tóth Bálint, Zainkó Csaba



Links:

[keras-tuner] https://github.com/keras-team/keras-tuner

[blog post] https://www.mikulskibartosz.name/using-keras-tuner-to-tune-hyperparameters-of-a-tensorflow-model/

In [ ]:
# az óra második felében a keras-tuner-t vizsgáljuk,
# ami 2019-ben vált elérhetővé (https://twitter.com/fchollet/status/1189992078991708160?lang=en),
# és most is aktívan fejlesztik,
# a legújabb commit-ok néhány naposak (https://github.com/keras-team/keras-tuner/commits/master)
# de mindenképp érdemes foglalkozni vele,
# mert nagy erőkkel dolgozik rajta a keras csapata

# néhány tutorial:
# 1) https://www.mikulskibartosz.name/using-keras-tuner-to-tune-hyperparameters-of-a-tensorflow-model/
# 2) https://www.mikulskibartosz.name/using-hyperband-for-tensorflow-hyperparameter-tuning-with-keras-tuner/
# 3) https://www.mikulskibartosz.name/how-to-automaticallyselect-the-hyperparameters-of-a-resnet-neural-network/

# a keras-tuner-hez tensorflow2.0 szükséges, 
%tensorflow_version 2.x 

In [ ]:
# és aztán telepítsük magát a keras-tuner-t is

!pip install keras-tuner

In [ ]:
# adatok betöltése ugyanúgy, mint eddig

import tensorflow as tf
from tensorflow import keras
import numpy as np

fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)


In [3]:
# először készítünk egy modell-generáló függvényt,
# ami a hp. hiperparaméterekből hálót állít elő
# a hyperas-hoz hasonlóan a keras-tuner-ben is különböző képpen lehet megadni a 
# hiperparaméter tartományokat:
# - hp.Int -> egész számok egy adott tartományban szétosztva, pl. konvolúciós filterek
# - hp.Choice -> választási lehetőség egy listából, pl. optimizáló
# - hp.Float -> az Int-hez hasonlóan kell min-max tartomány, pl. dropout-hoz jó
# - hp.Boolean -> bináris döntés, pl. egy adott háló-ág szerepeljen-e
# - hp.Fixed -> ha egy paramétert nem szeretnénk változtatni. erről majd később.
# részletek itt: https://github.com/keras-team/keras-tuner/blob/master/kerastuner/engine/hyperparameters.py


# most egy konvolúciós hálót rakunk össze a fashionmnist osztályozásra

def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=64, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  

  # when using the 'sparse_categorical_crossentropy' loss, your targets should be integer targets.
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model



In [4]:
# a legegyszerűbb hiperparaméter kereső algoritmus a randomsearch.
# a nevének megfelelően véletlenül választ a paraméterek közül
# most nem foglalkozunk vele, mert van ennél érdekesebb is

from kerastuner.tuners import RandomSearch

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    directory='output',
    project_name='FashionMNIST_random')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """


In [ ]:
# a keras-tuner-ben egyelőre nincs implementálva a TPE
# helyette van GP, ami a kerastuner.tuners.bayesian-ban elérhető
# eredménye hasonló a TPE-hez, ezért nem foglalkozunk vele külön

In [5]:
# helyette van Hyperband, ami először 'belenéz' többféle hálóba,
# és a rosszul teljesítő hálókat eldobja (pruning / metszés)
# 
# azaz futtat pl. 2-2 epoch-ot különböző hiperparaméter kombinációk közül,
# (a lenti log-ban: initial_epoch: 0)
# aztán a jól teljesítők közül megint 2-2 epoch,
# (a lenti log-ban: initial_epoch: 2)
# és így tovább, iteratívan szűkíti a keresési teret
# (a lenti log-ban: initial_epoch: 4, ...)
# a vége felé pedig már végigmegy az összes epoch-on
#
# a 'factor' paraméterrel lehet szabályozni, hogy mennyire gyorsan szűkítsen,
# a 'max_epochs' pedig a nevének megfelelően max ennyi epoch-ot enged
#

from kerastuner.tuners import Hyperband

tuner = Hyperband(
    build_model,
    objective='val_accuracy',
    factor=3,
    max_epochs=10,
    directory='output',
    project_name='FashionMNIST_hyperband')

In [6]:
# először nézzük meg, hogy mi lesz a keresési terünk

tuner.search_space_summary()

Search space summary
Default search space size: 6
conv_1_filter (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 128, 'step': 16, 'sampling': None}
conv_1_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
conv_2_filter (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 64, 'step': 16, 'sampling': None}
conv_2_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
dense_1_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001], 'ordered': True}


In [ ]:
# a tuner.search paraméterei a keras model.fit-hez hasonlóak
# ez végzi magát a hiperparaméter optimalizálást
# ha elindítjuk, kb. 15 percig fut

tuner.search(train_images, train_labels, epochs=10, validation_split=0.1)



Trial 21 Complete [00h 01m 22s]
val_accuracy: 0.9104999899864197

Best val_accuracy So Far: 0.9168333411216736
Total elapsed time: 00h 17m 59s

Search: Running Trial #22

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |80                |96                
conv_1_kernel     |3                 |5                 
conv_2_filter     |64                |48                
conv_2_kernel     |3                 |3                 
dense_1_units     |80                |128               
learning_rate     |0.01              |0.001             
tuner/epochs      |4                 |10                
tuner/initial_e...|0                 |4                 
tuner/bracket     |1                 |2                 
tuner/round       |0                 |2                 

Epoch 1/4
 130/1688 [=>............................] - ETA: 17s - loss: 1.0664 - accuracy: 0.6724

In [ ]:
# utána a tuner-ből kinyerhetjük a legjobb modellt, és azt használhatjuk tovább,
# pl újrataníthatunk vele

best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

# best_model.fit(train_images, train_labels, epochs=20, validation_split=0.1, initial_epoch=4)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        27696     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               3096688   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 3,126,154
Trainable params: 3,126,154
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# a tuner-ből kinyerhető, hogy mik voltak a legjobb hiperparaméterek

params_best = tuner.get_best_hyperparameters(num_trials=1)[0]
params_best.get_config()['values']

{'conv_1_filter': 64,
 'conv_1_kernel': 3,
 'conv_2_filter': 48,
 'conv_2_kernel': 3,
 'dense_1_units': 112,
 'learning_rate': 0.001,
 'tuner/bracket': 2,
 'tuner/epochs': 10,
 'tuner/initial_epoch': 4,
 'tuner/round': 2,
 'tuner/trial_id': 'f3d57c2f47ce641d7bfb50fae5051b05'}

In [ ]:
# a tuner-ből kinyerhető, hogy mik lettek a legjobb eredmények
tuner.results_summary()

In [ ]:
# konklúzió: konvolúciós háló + keras-tuner + hyperband: megint sikerült picit jobb eredményt elérni

In [ ]:
# a keras-tuner-ben megtalálható néhány nevezetes hálónak a hiperoptolható változata,
# konkrétan eddig Xception és ResNet van

# itt most nem előretanított hálót töltünk be, mint a transfer learning esetén,
# hanem a háló szerkezetét, amiben néhány paramétert optimalizálhatunk
# a saját adataink függvényében

# ahhoz, hogy megnézzük, mik a változtatható paraméterek, célszerű megnézni a hálók forrását,
# https://github.com/keras-team/keras-tuner/blob/master/kerastuner/applications/


In [ ]:
from kerastuner.tuners import Hyperband
from kerastuner.applications import HyperResNet
from kerastuner import HyperParameters

hypermodel = HyperResNet(input_shape=(28, 28, 1), classes=10)

hp = HyperParameters()
hp.Choice('learning_rate', values=[1e-3, 1e-4])
hp.Fixed('optimizer', value='adam')

'adam'

In [ ]:
tuner = Hyperband(
    hypermodel,
    objective='val_accuracy',
    hyperparameters=hp,
    tune_new_entries=False,
    max_epochs=5,
    directory='output',
    project_name='FashionMNIST_resnet')

In [ ]:
# nézzük meg, hogy mi lesz a keresési terünk

# mivel az optimizer-t fix-re állítottuk, azt nem fogja változtatni,
# és a 'tune_new_entries=False' miatt a többi hiperparamétert sem piszkálja
# csak a két learning_rate értéket fogja megnézni

tuner.search_space_summary()

In [ ]:
# ha beállítjuk, hogy 'tune_new_entries=False', akkor
# viszont a háló többi paraméterét is végignézné

tuner_large = Hyperband(
    hypermodel,
    objective='val_accuracy',
    hyperparameters=hp,
    # ez most True
    tune_new_entries=True,
    max_epochs=5,
    directory='output',
    project_name='FashionMNIST_resnet')

tuner_large.search_space_summary()

In [ ]:
# a ResNet-hez a címkéket onehot-enkódolni kell

from keras.utils import to_categorical
train_labels_binary = to_categorical(train_labels)

Using TensorFlow backend.


In [ ]:
# itt most csak 2 tanítás megy végig a 2-féle learning rate-tel
# de mivel a ResNet hálózat nagy, ezért sokáig tart

tuner.search(train_images, train_labels_binary, validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/2
54000/54000 [==============================] - 83s 2ms/sample - loss: 0.5668 - accuracy: 0.8071 - val_loss: 0.4473 - val_accuracy: 0.8313
Epoch 2/2
54000/54000 [==============================] - 70s 1ms/sample - loss: 0.4696 - accuracy: 0.8421 - val_loss: 0.3488 - val_accuracy: 0.8702


Train on 54000 samples, validate on 6000 samples
Epoch 1/2
54000/54000 [==============================] - 79s 1ms/sample - loss: 0.6895 - accuracy: 0.7522 - val_loss: 0.4961 - val_accuracy: 0.8193
Epoch 2/2
54000/54000 [==============================] - 71s 1ms/sample - loss: 0.4285 - accuracy: 0.8436 - val_loss: 0.4022 - val_accuracy: 0.8550


INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

In [ ]:
params_best = tuner.get_best_hyperparameters(num_trials=1)[0]
params_best.get_config()['values']

{'conv3_depth': 4,
 'conv4_depth': 6,
 'learning_rate': 0.001,
 'optimizer': 'adam',
 'pooling': 'avg',
 'tuner/bracket': 1,
 'tuner/epochs': 2,
 'tuner/initial_epoch': 0,
 'tuner/round': 0,
 'version': 'v2'}

In [ ]:
# a legjobb modellt visszaállíthatjuk a hypermodel és a params_best kombinációjából
model_best = tuner.hypermodel.build(params_best)

In [ ]:
print(model_best.summary())

Model: "ResNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 34, 34, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 14, 14, 64)   3200        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 16, 16, 64)   0           conv1_conv[0][0]                 
_____________________________________________________________________________________________

In [ ]:
# itt a gyakorlat vége
# konklúzió a keras-tuner-ről: viszonylag új, aktívan fejlesztett rendszer